<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_03_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- BIO1173_CLASS_03_2 -->

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

##### **Module 3: Convolutional Neural Networks (CNN's)**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Biology, Health and the Environment](https://sciences.utsa.edu/bhe/), [UTSA](https://www.utsa.edu/)

### Module 3 Material

* Part 3.1: Using Convolutional Neural Networks
* **Part 3.2: Using Pre-Trained Neural Networks with PyTorch**
* Part 3.3: Facial Recognition and Analysis
* Part 3.4: Introduction to GAN's for Image and Data Generation

#### **Change your Runtime Now!**

For this lesson you must have a GPU hardware accelerator (e.g. `A100` if available).

## Google CoLab Instructions

You MUST run the following code cell to get credit for this class lesson. By running this code cell, you will map your GDrive to /content/drive and print out your Google GMAIL address. Your Instructor will use your GMAIL address to verify the author of this class lesson.

In [ ]:
# @title You MUST Run This Cell First
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    Colab = True
    print("Note: Using Google CoLab")
    !curl ipinfo.io
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("**WARNING**: Your GMAIL address was **not** printed in the output below.")
    print("**WARNING**: You will NOT receive credit for this lesson.")
    Colab = False

You should see the following output except your GMAIL address should appear on the last line.

```text
Mounted at /content/drive
Note: Using Google CoLab
{
  "ip": "34.19.16.177",
  "hostname": "177.16.19.34.bc.googleusercontent.com",
  "city": "The Dalles",
  "region": "Oregon",
  "country": "US",
  "loc": "45.5946,-121.1787",
  "org": "AS396982 Google LLC",
  "postal": "97058",
  "timezone": "America/Los_Angeles",
  "readme": "https://ipinfo.io/missingauth"
}studentbio1173@gmail.com
``
If your GMAIL address does not appear. **Electronic Submission** will not process your Colab notebook.

# **----->TIME ALERT!<-----**

This lesson will probably **require close to 3 hours to complete**. Besides the normal issues there are two instances in which you are required to train a neural network. Training time for both neural networks is about 1 hour.

Don't start working on this lesson if you don't have sufficient free time to finish it.

# Accelerated Run-time Check

You MUST run the following code cell to get credit for this class lesson. The code in this cell checks what hardware acceleration you are using. To run this lesson, you must be running a Graphics Processing Unit (GPU).

In [ ]:
# @title Accelerated Run-time Check

import torch

# Check for GPU
def check_colab_gpu():
    print("=== Colab GPU Check ===")

    # Check PyTorch
    pt_gpu = torch.cuda.is_available()
    print(f"PyTorch GPU available: {pt_gpu}")

    if pt_gpu:
        print(f"PyTorch device count: {torch.cuda.device_count()}")
        print(f"PyTorch current device: {torch.cuda.current_device()}")
        print(f"PyTorch device name: {torch.cuda.get_device_name()}")
        print("You are good to go!")

    else:
        print("No compatible device found")
        print("WARNING: You must run this assigment using either a GPU to earn credit")
        print("Change your RUNTIME now and start over!")

check_colab_gpu()


If you current `Runtime` is correct you should see the following output

```text
=== Colab GPU Check ===
PyTorch GPU available: True
PyTorch device count: 1
PyTorch current device: 0
PyTorch device name: Tesla T4
You are good to go!
```

If your output is different, don't continue until change your `Runtime`.

# Create Custom Functions

The cell below creates custom function that are required to run this Colab notebook correctly.

If you fail to run this cell now, you will receive one (or more) error message(s) later in this lesson.

In [ ]:
# @title Create Custom Functions


# Simple function to print out elasped time
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

# Set random seed
def set_seed():
    """
    Sets the seed for reproducibility across Python, NumPy, and PyTorch.
    """
    import random

    # Set seed value
    seed_value=1173

    # 1. Base Python
    random.seed(seed_value)

    # 2. NumPy
    np.random.seed(seed_value)

    # 3. PyTorch (CPU and CUDA)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # For multi-GPU setups

    # 4. CuDNN Determinism (Crucial for GPU consistency)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # 5. Ensure all operations are deterministic
    torch.use_deterministic_algorithms(True)


print(f"✅ All custom functions have been created.")

If the code is correct, you should see the following output:
```
✅ All custom functions have been created.
```

### **YouTube Introduction to ResNet**

Run the next cell to see short YouTube introduction to ResNet. This is a suggested, but optional, part of the lesson.

In [ ]:
from IPython.display import HTML
video_id = "nc7FzLiB_AY"

HTML(f"""
<iframe width="560" height="315"
  src="https://www.youtube.com/embed/{video_id}"
  title="YouTube video player"
  frameborder="0"
  allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture"
  allowfullscreen
  referrerpolicy="strict-origin-when-cross-origin"> </iframe>
""")

# **DOWNLOAD AND INSTALL PRE-TRAINED NEURAL NETWORKS**

We will be using two pre-trained neural networks in this lesson, `ResNet50` and `ResNet101`. Run the next couple of code cells to download these neural networks to your Colab environment.

# Download `ResNet50`

Run the code in the cell below to download the ResNet101 model.


In [ ]:
# @title Download ResNet50
import torch
import torchvision.models as models
from tqdm import tqdm

# Create a progress bar
with tqdm(total=1, desc="Downloading ResNet50") as pbar:
    # Download ResNet50 with pre-trained weights
    weights = models.ResNet50_Weights.DEFAULT
    ResNet50_model_244 = models.resnet50(weights=weights)

    # Set the model to evaluation mode
    ResNet50_model_244.eval()

    # Update progress bar
    pbar.update(1)

# Print out results
print("\n✅ ResNet50 has been downloaded.")


If the code is correct, you should see the following output:
```text
Downloading ResNet50:   0%|          | 0/1 [00:00<?, ?it/s]Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth

  0%|          | 0.00/97.8M [00:00<?, ?B/s]
 18%|█▊        | 17.6M/97.8M [00:00<00:00, 183MB/s]
 44%|████▎     | 42.6M/97.8M [00:00<00:00, 230MB/s]
 69%|██████▉   | 67.2M/97.8M [00:00<00:00, 242MB/s]
100%|██████████| 97.8M/97.8M [00:00<00:00, 240MB/s]
Downloading ResNet50: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]
✅ ResNet50 has been downloaded.
```

# Download ResNet101

Run the code in the cell below to download the ResNet101 model.

In [ ]:
# @title Download ResNet101
import torch
import torchvision.models as models
from tqdm import tqdm

# Create a progress bar
with tqdm(total=1, desc="Downloading ResNet101") as pbar:
    # Download ResNet50 with pre-trained weights
    weights = models.ResNet101_Weights.DEFAULT
    ResNet101_model_512 = models.resnet101(weights=weights)

    # Set the model to evaluation mode
    ResNet101_model_512.eval()

    # Update progress bar
    pbar.update(1)

# Print out results
print("\n✅ ResNet101 has been downloaded.")

If the code is correct, you should see the following output:
```text
Downloading ResNet101:   0%|          | 0/1 [00:00<?, ?it/s]Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth

  0%|          | 0.00/171M [00:00<?, ?B/s]
 14%|█▎        | 23.2M/171M [00:00<00:00, 243MB/s]
 28%|██▊       | 48.2M/171M [00:00<00:00, 254MB/s]
 43%|████▎     | 73.8M/171M [00:00<00:00, 260MB/s]
 58%|█████▊    | 98.6M/171M [00:00<00:00, 257MB/s]
 72%|███████▏  | 123M/171M [00:00<00:00, 255MB/s]
100%|██████████| 171M/171M [00:00<00:00, 257MB/s]
Downloading ResNet101: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it]
✅ ResNet101 has been downloaded.
````

# Install Helper Functions for Examples and **Exercises**

The code in the cell below creates a two functions that we will need to use classify images in Examples 1 and 2.

* **make_square()** Since MobileNet is designed to classify images with the same number of horizontal and vertical pixels (i.e. a 'square' image), this function uses a combination of padding and cropping to convert any image into a 'square` image.

* **classify_image()** This function does most of the work. It first retrives the image from the HTTPS server and resizes it before processing it by the `ResNet50 model` that we previously downloaded. The actual prediction is made by this specific line of code:
```python
    prediction = ResNet_model_244(batch).squeeze(0)
```

In [ ]:
# @title  Install Helper Functions for Examples and Exercises

import torch
import torchvision.transforms as T
import torchvision.models as models
from PIL import Image, ImageFile, UnidentifiedImageError
import requests
import numpy as np
from io import BytesIO
from IPython.display import display

# ----------------------------------------------------------------------
# Global settings
# ----------------------------------------------------------------------
IMAGE_SIZE = 224
ImageFile.LOAD_TRUNCATED_IMAGES = False

# Load weights and model
weights = models.ResNet50_Weights.DEFAULT
ResNet_model_244 = models.resnet50(weights=weights)
ResNet_model_244.eval() # Set to inference mode

# Access the category labels and the specific preprocessing for these weights
categories = weights.meta["categories"]
preprocess = weights.transforms()

# Base URL of the images
ROOT = "https://biologicslab.co/BIO1173/images/class_03/"

# ----------------------------------------------------------------------
# Utility functions
# ----------------------------------------------------------------------
def make_square(img):
    """Crop the image to a square (center‑aligned)."""
    width, height = img.size
    side = min(width, height)
    left   = (width  - side) // 2
    top    = (height - side) // 2
    right  = left + side
    bottom = top  + side
    return img.crop((left, top, right, bottom))

# ----------------------------------------------------------------------
# Core function
# ----------------------------------------------------------------------
def classify_image(url):
    """
    Download an image, preprocess with Torchvision transforms,
    run through ResNet‑50, and display results.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content)).convert('RGB')
    except Exception as e:
        print(f"Error: {e}")
        return

    # 1. Standardize the image (optional: use make_square first)
    img_square = make_square(img)

    # 2. Apply the weight-specific preprocessing (Resizing, Normalizing, and Tensor conversion)
    # PyTorch expects a batch dimension: [Batch, Channels, Height, Width]
    batch = preprocess(img_square).unsqueeze(0)

    # 3. Predict
    with torch.no_grad(): # Disable gradient calculation for efficiency
        prediction = ResNet_model_244(batch).squeeze(0)
        # Apply Softmax to get probabilities
        probs = torch.nn.functional.softmax(prediction, dim=0)

    # 4. Get Top-5
    top5_prob, top5_catid = torch.topk(probs, 5)

    # Show the image
    display(img_square.resize((IMAGE_SIZE, IMAGE_SIZE)))

    # Print the top‑5 predictions
    print("\nTop‑5 predictions:")
    for i in range(top5_prob.size(0)):
        label = categories[top5_catid[i]]
        score = top5_prob[i].item()
        print(f"  {label:<25} : {score*100:5.2f}%")

# Print out results
print(f"✅ Helper functions have been created.")

If the code is correct, you should see the following output:
```text
✅ Helper functions have been created.
```


# Example 1: Classify Images with ResNet50

The code in the cell below downloads an image of a dog from the course fileserver, https://biologicslab.co and then uses the pre-trained `ResNet50` neural network to to classify it.

The image name is "pembroke_corgi.jpg"

In [ ]:
# @title Example 1: Classify Image with ResNet50

# Enter image name
image_name = "pembroke_corgi.jpg"

# Generate image path using the ROOT variable defined earlier
image_path = ROOT + image_name

# Print path
print(f"Analyzing {image_path}\n")

# Use the PyTorch-based classify_image function
classify_image(image_path)

If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/pembroke_corgi.jpg)
```text
Top‑5 predictions:
  Pembroke                  : 35.55%
  Cardigan                  :  5.37%
  Bernese mountain dog      :  0.32%
  collie                    :  0.29%
  Shetland sheepdog         :  0.24%
```

`ResNet50` has been trained to recognize a wide range of common objectes. Here is a basic list:

| Domain                           | Example classes (just a few from each)                                   |
|----------------------------------|--------------------------------------------------------------------------|
| Animals – mammals                | chimpanzee, tiger, lion, zebra, elephant                                |
| Animals – birds                  | eagle, sparrow, penguin, flamingo, crane                                 |
| Animals – reptiles & amphibians  | alligator, snake, frog, turtle                                          |
| Animals – fish & crustaceans     | goldfish, salmon, shrimp, lobster                                       |
| Animals – insects                | bee, butterfly, ant, dragonfly                                          |
| Plants & flowers                 | daisy, sunflower, orchid, cactus                                        |
| Fruits & vegetables              | apple, orange, broccoli, carrot                                         |
| Vegetables & nuts                | potato, tomato, almond, cashew                                          |
| Instruments & musical equipment  | guitar, piano, violin, saxophone                                        |
| Sports & equipment               | tennis racket, golf ball, basketball, soccer ball                       |
| Vehicles – land                  | car, truck, motorcycle, bicycle                                         |
| Vehicles – air                   | airplane, helicopter, jet, glider                                       |
| Vehicles – water                 | boat, ship, submarine, ferry                                            |
| Buildings & architecture         | bridge, house, church, skyscraper                                       |
| Furniture                        | chair, table, sofa, bed                                                  |
| Home appliances                  | microwave, refrigerator, toaster, blender                               |
| Tools & hardware                 | hammer, screwdriver, wrench, drill                                      |
| Clothing & accessories           | t‑shirt, hat, shoes, glasses                                            |
| Food & drinks                    | coffee, tea, pizza, cake                                                |
| Miscellaneous                    | toilet paper, keyboard, watch, trophy                                   |


Clearly `ResNet50` was trained to classify dogs. What is somewhat interesting, is that `ResNet50` appears to be quite good as to correctly identify a dog's breed. `ResNet50` was absolutedly correct that the image showed a `Welsh Pembroke Corgi`.

The **Pembroke Welsh Corgi** is a spirited, compact herding dog originally from Wales. Known for its short legs, fox-like ears, and expressive eyes, it's affectionate, intelligent, and highly trainable. Pembrokes thrive on activity, enjoy family life, and are renowned for their loyal, playful nature and distinctive “corgi grin.”

### **Exercise 1: Classify Images with ResNet50**

In the cell below write the code to download an another dog image from the course fileserver:

```type
image_name="bouvier_des_flandres.jpg"
```
and then uses `ResNet50` to classify it. This species is much less common than Corgis.

The **Bouvier des Flandres**, or **Bouvier de Flanders**, is a muscular, medium‑to‑large herding dog originally bred in the Flemish region of Belgium to manage cattle, sheep and pack loads. With a dense, double‑coated coat that comes in black, brown, red or tricolor, they are built for endurance and can thrive in both wet and dry climates. Their temperament is confident and affectionate, yet they possess a strong work ethic and a naturally protective instinct, making them excellent companion animals as well as valuable in search‑and‑rescue, therapy, and police work. Bouviers are intelligent and trainable, but they require consistent socialization and mental stimulation to prevent stubbornness or frustration. Health concerns are relatively few—chiefly hip dysplasia, gastric dilatation‑volvulus (bloat) and certain eye conditions—so with proper care, they can live 10–12 years.

In [ ]:
# Insert your code for Exercise 1 here

# Enter image name
image_name = "bouvier_des_flandres.jpg"

# Generate image path
image_path = ROOT + image_name

# Print path
print(f"Analyzing {image_path}")

# Use the PyTorch-based classify_image function
classify_image(image_path)

If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/bouvier_des_flandres.jpg)
```text
Top‑5 predictions:
  Bouvier des Flandres      : 25.45%
  Irish water spaniel       : 14.09%
  briard                    :  2.02%
  Tibetan terrier           :  1.59%
  miniature poodle          :  1.37%
```

Again, `ResNet50` got the specific dog breed correct.

# Example 2: Classify Retinal Image with ResNet50

What about medical image data? Can `ResNet50` analyze a **color fundus photograph** of the interior surface of a human retina?

Run the code in the Example 2 to see how `ResNet50` does with the following retinal image:

```text
"Retina_Score_0.png"
```


In [ ]:
# @title Example 2: Classify Retinal Image with ResNet50

# Enter image name
image_name = "Retina_Score_0.png"

# Generate image path
image_path = ROOT + image_name

# Print path
print(f"Analyzing {image_path}")

# Use the PyTorch-based classify_image function
classify_image(image_path)

If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/Retina_Score_0.png)
```text
Top‑5 predictions:
  chambered nautilus        : 33.39%
  jellyfish                 : 10.28%
  lampshade                 :  2.86%
  nematode                  :  2.79%
  butternut squash          :  1.25%
```

As expected, `ResNet50` was not trained to recognize retinal images. For this particular retinal image, `ResNet50` best guess (33.39% probability) that this image was a chambered nautilus. Here is one image of a chambered nautilus that is vaguely similar:

![___](https://biologicslab.co/BIO1173/images/class_03/chambered_nautilus.png)


# **Exercise 2: Classify Retinal Image with `ResNet50`**

In the cell below write the code to analyze the another retinal color fundus image:
```text
"Retina_Score_1.png"
```


In [ ]:
# @title Exercise 2: Classify Retinal Image with ResNet50

# Enter image name
image_name = "Retina_Score_1.png"

# Generate image path
image_path = ROOT + image_name

# Print path
print(f"Analyzing {image_path}")

# Use the PyTorch-based classify_image function
classify_image(image_path)

If the code is correct you should see the following output:

![___](https://biologicslab.co/BIO1173/images/class_03/Retina_Score_1.png)

```text
Top‑5 predictions:
  chambered nautilus        : 16.00%
  shield                    : 10.03%
  gong                      :  3.76%
  jellyfish                 :  3.72%
  jack-o'-lantern           :  2.81%
```

Once again, it would be charitable to say that `ResNet50` was a little weak when it comes to analyzing clinical images.

# **Transfer Learning for Computer Vision**

## **`ResNet`**

Many advanced prebuilt neural networks are available for computer vision, and Keras provides direct access to many networks. **Transfer Learning** is the technique where you use these prebuilt neural networks.

There are several different levels of transfer learning.

* Use a prebuilt neural network in its entirety
* Use a prebuilt neural network's structure
* Use a prebuilt neural network's weights

In this lesson we will use a popular prebuilt CNN called **`ResNet (Residual Network)`** built by Microsoft Research in 2015. The name comes from the fact that this network was designed to address the **vanishing gradient problem** that occurs when training very deep neural networks.

Instead of learning the direct mapping from input to output, `ResNet` learns the residual (i.e., the difference between the input and the output).
This is achieved using **skip connections** (also called shortcut connections), which allow the input to bypass one or more layers and be added directly to the output.

## **Transfer Learning**

**Transfer learning** is a machine learning technique where a model developed for one task is reused as the starting point for a model on a second task. In the context of image recognition, this approach is particularly effective when using deep convolutional neural networks (CNNs) like **`ResNet`**.

### **Why Use Transfer Learning?**

Training deep neural networks from scratch requires large datasets and significant computational resources. Transfer learning mitigates this by leveraging pre-trained models—typically trained on large benchmark datasets like ImageNet—to extract general features from images. These features can then be fine-tuned for a specific task with a smaller, domain-specific dataset.

### **How `ResNet` Supports Transfer Learning**

`ResNet` is a widely used CNN architecture known for its use of **residual connections**, which help in training very deep networks by addressing the vanishing gradient problem. Pre-trained versions of `ResNet` (e.g., `ResNet-50`, `ResNet-101`) are commonly used as feature extractors in transfer learning workflows.

### **Benefits**

- **Reduced Training Time**: Leverages existing learned features.
- **Improved Performance**: Often achieves better accuracy with less data.
- **Flexibility**: Can be adapted to a wide range of image classification tasks.

Transfer learning with `ResNet` is a powerful and efficient approach for developing high-performing image recognition models, especially when data or computational resources are limited.


# Example 3: Improving `ResNet` Classification Accuracy of Diabetic Retinopathy

We know from from above that the native (base) `ResNet50` neural network is unable to effectively analyze clinical retinal images.

Example 3 will demonstrate how **transfer learning** can leverage `ResNet50` as a starting point to create a new neural network that can effectively classify retinal images as to their degree of diabetic retinopathy.  

#### **`Diabetic Retinopathy Image Dataset` Classes**

The dataset we will be using is called the **`Diabetic Retinopathy Image Dataset`**. This image dataset consists of color fundus photographs—high‑resolution RGB images of the interior surface of the eye (the retina).

These are standard clinical retinal images obtained with a fundus camera (usually a 45° or 50° field-of-view, non‑mydriatic or mydriatic camera) and capture the posterior pole (macula, optic disc, retinal vessels, and surrounding retinal tissue). The images are typically stored as JPEG/PNG files with dimensions on the order of several thousand pixels (e.g., 3500x2333 px) and are used for grading the severity of diabetic retinopathy.

The severity of diabetic retinopathy was divided into **five classes** as follows:
1. **Class 0** - No Diabetic Retinopathy (No_DR)
2. **Class 1** - Mild Non-Proliferative Diabetic Retinopathy (NPDR)
3. **Class 2** - Moderate NPDR
4. **Class 3** - Severe NPDR
5. **Class 4** - Proliferative Diabetic Retinopathy (PDR)

An example of each class is shown below:

![__](https://biologicslab.co/BIO1173/images/class_03/DiabeticRetinopathyData.png)

It should be noted that differences in apparent retinal size has nothing to do with the degree of retinopathy but instead reflects the fact that the retinal images were taken by a large number of clinicians, with different imaging equipment and procedures. In other words, there is considerable "noise" in the image set.



### Use `torchvision` for Data Loading

For Example 3 and **Exercise 3**, we will use **Torchvision** to handle image loading and preprocessing. In PyTorch, image generation is handled by a combination of `torchvision.transforms` (for image manipulation) and `torchvision.datasets` (for loading images from directories).

Since `torchvision` is part of the standard PyTorch installation, no additional package installs are usually required.

In [ ]:
# Install PyTorch and Torchvision
!pip install -q torch torchvision

If the code is correct you should _not_ see any output


# Example 3 - Step 1: Set ENVIRONMENTAL VARIABLES

The code in the cell below defines a number of `ENVIRONMENTAL VARIABLES` that are needed for later code cells.

The use of **Enivornment Variables** can allow code to be **configurable without modification**. For example, you might use different database URLs for development, testing, and production environments. As you will see later, you will re-use this code cell for **Exercise 3 - Step 1** below.

In [ ]:
# @title Example 3 - Step 1: Set ENVIRONMENTAL VARIABLES

import os

# ------------------------------------------------------------------------
# 1️⃣  Create variables for downloading loading Zip file
# ------------------------------------------------------------------------
URL = "https://biologicslab.co/BIO1173/data/"
DOWNLOAD_SOURCE = URL+"diabetic_retinopathy_train_244.zip"
DOWNLOAD_NAME = DOWNLOAD_SOURCE[DOWNLOAD_SOURCE.rfind('/')+1:]
print("DOWNLOAD_SOURCE=",DOWNLOAD_SOURCE)
print("DOWNLOAD_NAME=",DOWNLOAD_NAME)

# ------------------------------------------------------------------------
# 2️⃣  Create variables for extracting the Zip file
# ------------------------------------------------------------------------
PATH = "./"
EXTRACT_TARGET = os.path.join(PATH,"retinopathy_244")
SOURCE = os.path.join(EXTRACT_TARGET, "train_244")
print("EXTRACT_TARGET=",EXTRACT_TARGET)
print("SOURCE=",SOURCE)

# ------------------------------------------------------------------------
# 3️⃣  Print variables for debugging
# ------------------------------------------------------------------------
print("ENVIRONMENTAL VARIABLES were successfully created.")

If the code is correct you should see the following output
```text
DOWNLOAD_SOURCE= https://biologicslab.co/BIO1173/data/diabetic_retinopathy_train_244.zip
DOWNLOAD_NAME= diabetic_retinopathy_train_244.zip
EXTRACT_TARGET= ./retinopathy_244
SOURCE= ./retinopathy_244/train_244
ENVIRONMENTAL VARIABLES were successfully created.
```

# Example 3 - Step 2: Download and Extract Image Data

The code in the cell below downloads a zip file and extracts it. The names of the file server, zip file and folders were set above as **`ENVIRONMENTAL VARIABLES`** in the previous step.

**TIME ALERT:** Even when compressed (i.e. "zipped"), image data file are typically quite large so download and extraction times can often take a few minutes.

In [ ]:
# @title Example 3 - Step 2: Download and Extract Image Data

import os
import subprocess
import sys

# --------------------------------------------------------------
# 1️⃣  Create directories
# --------------------------------------------------------------
print("Creating necessary directories...", end='')
# Create necessary directories
os.makedirs(SOURCE, exist_ok=True)
os.makedirs(EXTRACT_TARGET, exist_ok=True)
print("done.")

# --------------------------------------------------------------
# 2️⃣  Download Zip file
# --------------------------------------------------------------
print(f"Downloading {DOWNLOAD_NAME}...", end='')
# Define paths and URLs
download_path = os.path.join(PATH, DOWNLOAD_NAME)
extract_path = os.path.join(EXTRACT_TARGET, DOWNLOAD_NAME)
# Download the file
try:
    result = subprocess.run(
        ["wget", "-O", DOWNLOAD_NAME, DOWNLOAD_SOURCE],
        check=True,
        capture_output=True,
        text=True
    )
    print("done.")
except subprocess.CalledProcessError as e:
    print(f"Download failed with error: {e}")
    print(f"Error output: {e.stderr}")
    sys.exit(1)

# --------------------------------------------------------------
# 3️⃣  Extract Zip file
# --------------------------------------------------------------
print(f"Extracting {DOWNLOAD_NAME} to {EXTRACT_TARGET}...", end='')

# Check if zip file exists and has content
if not os.path.exists(DOWNLOAD_NAME):
    print(f"Error: Zip file {DOWNLOAD_NAME} does not exist")
    sys.exit(1)

if os.path.getsize(DOWNLOAD_NAME) == 0:
    print(f"Error: Zip file {DOWNLOAD_NAME} is empty")
    sys.exit(1)

# Extract the file with error handling
try:
    # Use -o flag to overwrite files without prompting
    # Use -q for quiet mode
    result = subprocess.run(
        ["unzip", "-o", "-q", DOWNLOAD_NAME, "-d", EXTRACT_TARGET],
        check=True,
        capture_output=True,
        text=True
    )
    print("done.")
except subprocess.CalledProcessError as e:
    print(f"Error: Unzipping failed with return code {e.returncode}")
    print(f"Error output: {e.stderr}")
    print(f"Command that failed: {e.cmd}")
    sys.exit(1)
except FileNotFoundError:
    print("Error: unzip command not found. Please install unzip.")
    sys.exit(1)

# --------------------------------------------------------------
# 4️⃣  Verify Extraction was successful
# --------------------------------------------------------------
print("Verifying Extraction...")
try:
    # Check if extraction directory exists
    if not os.path.exists(EXTRACT_TARGET):
        print(f"Error: Extraction directory {EXTRACT_TARGET} does not exist")
        sys.exit(1)

    # List contents to verify
    contents = os.listdir(EXTRACT_TARGET)
    if len(contents) == 0:
        print("Warning: Extraction directory is empty")
    else:
        print(f"Successfully extracted {len(contents)} items:")
        for item in sorted(contents)[:10]:  # Show first 10 items
            item_path = os.path.join(EXTRACT_TARGET, item)
            if os.path.isfile(item_path):
                size = os.path.getsize(item_path)
                print(f"  - {item} ({size} bytes)")
            else:
                print(f"  - {item} (directory)")

        if len(contents) > 10:
            print(f"  ... and {len(contents) - 10} more items")

    # Try to get more detailed information about what was extracted
    result = subprocess.run(
        ["ls", "-la", EXTRACT_TARGET],
        capture_output=True,
        text=True,
        check=False  # Don't raise exception for this command
    )
    if result.returncode == 0:
        print("Directory contents:")
        print(result.stdout)

except Exception as e:
    print(f"Error during verification: {e}")
    sys.exit(1)

print("Extraction completed successfully!")


If the code is correct, you should see the following output:
```text
Creating necessary directories...done.
Downloading diabetic_retinopathy_train_244.zip...done.
Extracting diabetic_retinopathy_train_244.zip to ./retinopathy_244...done.
Verifying Extraction...
Successfully extracted 2 items:
  - trainLabels.csv (465317 bytes)
  - train_244 (directory)
Directory contents:
total 1040
drwxr-xr-x 3 root root   4096 Mar  1 01:53 .
drwxr-xr-x 1 root root   4096 Mar  1 01:52 ..
drwxr-xr-x 2 root root 581632 Mar  1 01:53 train_244
-rw-rw---- 1 root root 465317 Sep 12 15:05 trainLabels.csv

Extraction completed successfully!
```



# Example 3 - Step 3: Load Labels for the Training Set

CNNs are trained by _supervised_ learning, which means that during each training step the network's prediction must be compared to a known correct answer. The label supplies this target: it allows a loss function (e.g., cross‑entropy, IoU, etc.) to be computed, which in turn provides gradients that tell the optimizer how to adjust the weights. Without a label, the loss cannot be evaluated, no gradients can be derived, and the network has no signal to improve its predictions. Thus, labels are essential for defining the training objective and enabling the network to learn from data.

The file `trainLabels.csv` contains the label information for our retinal images. This file has just two columns, **image** and **level**. The `image` specifies the image's filename and from which eye the image was obtained; for example, `10_left.png`. The `level` column contains the a numerical value between 0 and 4 which indicates the serverity of diabetic retinopathy. So in this example the **_level_** is the **image label**.

The code in the cell below reads the file `trainLabels.csv` and creates a Pandas dataframe called `eg_raw_df` to store the label information. A short amount of `eg_raw_df` is printed out for inspection to make sure the code worked as expected.

In [ ]:
# @title Example 3 - Step 3: Load the Labels for the Training Set

import pandas as pd

# Read labels and create dataframe
eg_raw_df = pd.read_csv(
        os.path.join(EXTRACT_TARGET,"trainLabels.csv"),
        na_values=['NA', '?'])

# Add file extention
image_col = 'image'
eg_raw_df[image_col] = eg_raw_df[image_col].astype(str) + '.png'

# Print sample for verification
eg_raw_df

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image01A.png)

# Example 3 - Step 4: Validate Images

The code in the cell below checks to see if there is an actual retinal image for each image label in contained the DataFrame `eg_raw`, that was generated in the previous step.  

In [ ]:
# @title Example 3 - Step 4: Validate Images

from pathlib import Path

# Validate that all image files actually exist
def validate_image_files(df, source_path):
    source = Path(source_path)
    existing_files = set()

    # Get all actual files in the directory
    for file_path in source.rglob('*'):
        if file_path.is_file() and file_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
            existing_files.add(file_path.name)

    # Check which files in DataFrame actually exist
    df['file_exists'] = df['image'].apply(lambda x: x in existing_files)

    print(f"Total images in DataFrame: {len(df)}")
    print(f"Images that exist: {df['file_exists'].sum()}")
    print(f"Missing images: {(~df['file_exists']).sum()}")

    # Filter to only include existing files
    valid_df = df[df['file_exists']].copy()
    print(f"Valid DataFrame size: {len(valid_df)}")

    return valid_df

# Use it:
eg_raw_df = validate_image_files(eg_raw_df, SOURCE)

If the code is correct you should see the following output
```text
Total images in DataFrame: 35126
Images that exist: 17448
Missing images: 17678
Valid DataFrame size: 17448
```

# Example 3 - Step 5: Split Images into Training and Validation Sets

If you judge the model only on the same data you used to update its weights, you’ll get an *optimistic* estimate of its performance – the model will look perfect on the training set but will usually fail on new images.  
Splitting the data into **training + validation** lets you:

1. **Measure true generalization** - see how the network behaves on unseen samples.  
2. **Tune hyper‑parameters** - learning rate, number of layers, data‑augmentation policies, etc.  
3. **Detect & prevent over‑fitting** - monitor validation loss/accuracy during training and stop or adjust when the model starts to degrade.  
4. **Select the best model** - keep the checkpoint that had the lowest validation error.  

Without a validation set, you'll be blind to over-fitting, you'll have no reliable metric for early stopping, and you'll have no principled way to pick the best architecture or hyper-parameters.

The code in the cell below, splits the retinal images into a training set and a validation set. How much of the train set is used for the validation set depends on the variable `FRAC`. In the cell below, `FRAC` is set to 0.8 which means 80% of the images will be put into the training set (`eg_train_df`) and the remaining 20% will be put into the validation set (`eg_val_df`). Which images are used in each set is randomize. It should be noted that only the **image names** are being "split" into a training and validation pool. Later an image generator will use these two DataFrames to actually generate two image sets.

After the split, the number images assigned to both sets is printed out.

In [ ]:
# @title Example 3 - Step 5: Split Images into Training and Validation Sets

# Set split fraction
FRAC=0.8  # 80% training / 20% validation

# Convert the class column to string – required for `flow_from_dataframe`
eg_raw_df['level'] = eg_raw_df['level'].astype(str)

# ------------------------------------------------------------------
#  1️⃣ Randomly split data into training and validation sets
# ------------------------------------------------------------------
eg_train_df = eg_raw_df.sample(frac=FRAC, random_state=42)
eg_val_df   = eg_raw_df.drop(eg_train_df.index)

# Calculate the split fraction as sanity check
split_fraction = len(eg_train_df) / (len(eg_val_df) + len(eg_train_df))

# Print out numbers
print(f"Training set size   : {len(eg_train_df)}")
print(f"Validation set size : {len(eg_val_df)}")
print(f"Calculated split fraction =", split_fraction)

# Quick sanity check
print("\nSample training rows:")
print(eg_train_df[['image', 'level']].head())

print("\nSample validation rows:")
print(eg_val_df[['image', 'level']].head())

If the code is correct, you should see the following output:
```text
Training set size   : 13958
Validation set size : 3490
Calculated split fraction = 0.7999770747363595

Sample training rows:
                 image level
16508   20710_left.png     0
8057   10109_right.png     2
23120   29217_left.png     0
24         31_left.png     0
12902   16217_left.png     0

Sample validation rows:
           image level
1   10_right.png     0
7   16_right.png     4
10   19_left.png     0
13  20_right.png     0
15  21_right.png     0
```

# Example 3 - Step 6: Final Check for Valid Images

The code in the cell below does one more final check for valid image files before image transforms occur and the dataloader is created in the next step.

In [ ]:
# @title Example 3 - Step 6: Final Check for Valid Images

import os
from pathlib import Path

# Check which files are actually valid
def check_valid_images(df, source_path):
    valid_files = []
    invalid_files = []

    for filename in df['image']:
        file_path = Path(source_path) / filename
        if file_path.exists() and file_path.is_file():
            valid_files.append(filename)
        else:
            invalid_files.append(filename)

    print(f"Total files: {len(df)}")
    print(f"Valid files: {len(valid_files)}")
    print(f"Invalid files: {len(invalid_files)}")

    # Remove invalid files from your dataframe
    df_clean = df[df['image'].isin(valid_files)]
    print(f"After cleaning: {len(df_clean)} samples")

    return df_clean

# Apply cleaning to both train and validation sets
eg_train_df = check_valid_images(eg_train_df, SOURCE)
eg_val_df = check_valid_images(eg_val_df, SOURCE)


If the code is correct you should see the following output

```text
Total files: 13958
Valid files: 13958
Invalid files: 0
After cleaning: 13958 samples
Total files: 3490
Valid files: 3490
Invalid files: 0
After cleaning: 3490 samples
```

# Example 3 - Step 7: Create Image Transforms and DataLoaders

The code in the cell below uses **torchvision.transforms** to define the sequence of augmentations and DataLoader to manage memory and batching. This separation provides more flexibility and control over the preprocessing pipeline.

**IMPORTANT NOTE:**

You need to be very careful to correctly set the **IMAGE SIZE**. This is especially true when performing transfer learning with pre-trained neural networks like `ResNet50`. `ResNet50` is designed to work with square images of exactly `244` pixels wide and `244` pixels high (i.e. `244 X 244`) that have 3 color channels (i.e. `RGB`) and no `alpha` channel.

In the cell below, here is the code that specifies the image size:

```type
# Specify image size
EG_IMG_W = 244
EG_IMG_H = 244
```
You should also note that we set the `batch` sizes for training at this step in process:
```text
# Specify batch size
EG_BATCH_TRAIN  = 64
EG_BATCH_VAL    = 64
```
If we need to use a different batch size we will need to go back and re-run this cell again.

You should also note that we only augment the **training images** --  the **validation images** are **not** augmented.

In [ ]:
# @title Example 3: Step 7: Create Image Transforms and DataLoaders

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from PIL import Image
import os

# ------------------------------------------------------------------
# 0️⃣ IMPORTANT: Specify image size and batch size
# ------------------------------------------------------------------
EG_IMG_W = 244
EG_IMG_H = 244
EG_BATCH_TRAIN = 64
EG_BATCH_VAL = 64

# ------------------------------------------------------------------
# 1️⃣ Define the Custom Dataset
# ------------------------------------------------------------------
class RetinopathyDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform
        # Create a mapping for categories if they are strings
        self.classes = sorted(self.df['level'].unique())
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.df.iloc[idx]['image'])
        image = Image.open(img_name).convert('RGB')

        # In PyTorch, labels are usually long integers (CrossEntropyLoss)
        # rather than one-hot encoded vectors.
        label = self.class_to_idx[self.df.iloc[idx]['level']]

        if self.transform:
            image = self.transform(image)

        return image, label

# ------------------------------------------------------------------
# 2️⃣ Define Transforms (Augmentation for Train, Clean for Val)
# ------------------------------------------------------------------
# PyTorch ResNet normalization values
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transforms = T.Compose([
    T.Resize((EG_IMG_H, EG_IMG_W)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(20),
    T.RandomAffine(degrees=0, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    T.ToTensor(),
    T.Normalize(norm_mean, norm_std)
])

val_transforms = T.Compose([
    T.Resize((EG_IMG_H, EG_IMG_W)),
    T.ToTensor(),
    T.Normalize(norm_mean, norm_std)
])

# ------------------------------------------------------------------
# 3️⃣ Create DataLoaders
# ------------------------------------------------------------------
train_dataset = RetinopathyDataset(eg_train_df, str(SOURCE), transform=train_transforms)
val_dataset = RetinopathyDataset(eg_val_df, str(SOURCE), transform=val_transforms)

eg_train_loader = DataLoader(train_dataset, batch_size=EG_BATCH_TRAIN, shuffle=True, num_workers=0)
eg_val_loader = DataLoader(val_dataset, batch_size=EG_BATCH_VAL, shuffle=False, num_workers=0)

# Sanity Check
eg_x_train, eg_y_train = next(iter(eg_train_loader))
eg_x_val, eg_y_val = next(iter(eg_val_loader))

print(f"TRAIN batch images: {eg_x_train.shape}") # [Batch, Channels, H, W]
print(f"TRAIN batch labels: {eg_y_train.shape}") # [Batch]
print(f"VAL batch images:   {eg_x_val.shape}")
print(f"VAL batch labels:   {eg_y_val.shape}")

If the code is correct, you should see the following output:

```text
TRAIN batch images: torch.Size([64, 3, 244, 244])
TRAIN batch labels: torch.Size([64])
VAL batch images:   torch.Size([64, 3, 244, 244])
VAL batch labels:   torch.Size([64])
```

# Example 3 - Step 8: Check Class Distribution

The code in the cell below generates a histogram showing the distribution of the 5 classes in the validation data set.

#### **Why Check Class Distribution in CNN Classification?**

When training a Convolutional Neural Network (CNN) for image classification, examining the distribution of classes in your dataset helps ensure that your model learns effectively and generalizes well. Here's why it's important:

##### **1. Detecting Class Imbalance**
If one class has significantly more samples than others, the model may become biased toward predicting the majority class. This can lead to:
- High accuracy but poor performance on minority classes.
- Misleading evaluation metrics.

##### **2. Choosing the Right Metrics**
In imbalanced datasets, accuracy alone is not a reliable metric. You may need to use:
- Precision, recall, F1-score
- Confusion matrix
- ROC-AUC (for binary classification)

##### **3. Designing Better Validation Strategies**
Knowing the class distribution helps in:
- Stratified sampling for train/test splits
- Ensuring each class is represented in validation and test sets

##### **4. Applying Corrective Techniques**
If imbalance is detected, you can apply:
- **Data augmentation** for minority classes
- **Class weighting** in the loss function
- **Oversampling** or **undersampling**
- **Synthetic data generation** (e.g., SMOTE)

##### **5. Improving Interpretability**
Understanding class distribution helps interpret model behavior and debug issues like:
- Why the model is misclassifying certain classes
- Why training loss is low but validation performance is poor


### **Best Practice**
Always visualize class distribution before training using a bar chart or value counts.

Following **Best Practice**, the code in the cell below generates a bar chart showing the class distribution before we start our training.

In [ ]:
# @title Example 3 - Step 8: Check Class Distribution (PyTorch)

import numpy as np
import matplotlib.pyplot as plt

# In PyTorch, we can pull the labels directly from the dataset's dataframe
# or use the class_to_idx mapping we created in the RetinopathyDataset
eg_labels_val = val_dataset.df['level'].values

# If labels are strings, map them to integers for bincount
label_map = val_dataset.class_to_idx
eg_labels_val_indices = [label_map[label] for label in eg_labels_val]

# Count class distribution
eg_class_counts = np.bincount(eg_labels_val_indices)

# Plot distribution
plt.figure(figsize=(8, 4))
plt.bar(range(len(eg_class_counts)), eg_class_counts, color='skyblue', edgecolor='black')

# Add class names to the X-axis for better readability
plt.xticks(range(len(eg_class_counts)), list(label_map.keys()))

plt.xlabel("Class Name (Condition Severity)")
plt.ylabel("Number of Samples")
plt.title("Validation Set Class Distribution")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Print counts for verification
for cls, idx in label_map.items():
    print(f"Class {idx} ({cls}): {eg_class_counts[idx]} samples")

If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image02A.png)

```text
Class 0 (0): 1602 samples
Class 1 (1): 516 samples
Class 2 (2): 1055 samples
Class 3 (3): 175 samples
Class 4 (4): 142 samples
```

The classes are not balanced. Healthy retinal images (Class `0`) outnumbers the other classes. Unfortunately this situation is quite common in medical imaging of pathological states. The problem is that our neural network model can quickly learn that if it picks `0` as the image label, it will be correct _most_ of the time. We need to keep this imbalance in mind when we interpret the results of training our neural network.

# Example 3 - Step 9: Setup Neural Network and Define Training Components

The code in the cell below demonstrates how to use **transfer learning** with the **ResNet50** architecture in PyTorch for a custom image classification task involving 5 classes (degrees of diabetic retinopathy).

#### **What It Does**

- **Loads a pretrained `ResNet50` model**: We load the model with weights trained on ImageNet.
- **Freezes the base model's weights**: We disable gradient calculations for the pre-trained layers to retain learned features and speed up training.
- **Modifies the "Head" (Final Layer)**:
  - In PyTorch, ResNet50 already has a Global Average Pooling layer.
  - We replace the final Fully Connected (`fc`) layer with a custom **Sequential** block:
    - **Linear Layer**: To transition from the base features (2048) to a hidden layer.
    - **ReLU Activation**: For non-linearity.
    - **Dropout**: For regularization to prevent overfitting.
    - **Final Linear Layer**: To output scores for our **5 classes**.
- **Defines the Loss Function and Optimizer**:
  - **CrossEntropyLoss**: The PyTorch standard for multi-class classification (combines LogSoftmax and NLLLoss).
  - **Adam Optimizer**: Specifically configured to only update the parameters of our new "head."

#### **Use Case**

This approach is ideal when:
- You have limited training data.
- You want to leverage powerful pretrained models.
- You need to adapt a general-purpose model to a specific classification task.

In the cell below, we create `ResNet50_model_244`. We first freeze all parameters in the base model. Then, we overwrite `model.fc` with our custom architecture. Finally, we move the model to the **GPU** (if available) to ensure high-performance training.

In [ ]:
# @title Example 3 - Step 9: Setup Neural Network and Define Training Components

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Set the seed
set_seed()

# 1️⃣ Load the base ResNet50 model with pre-trained weights
weights = models.ResNet50_Weights.DEFAULT
ResNet50_model_244 = models.resnet50(weights=weights)

# 2️⃣ Freeze all layers in the base model
for param in ResNet50_model_244.parameters():
    param.requires_grad = False

# 3️⃣ Modify the "Head"
# ResNet50 already has GlobalAveragePooling (called avgpool) before the fc layer.
# We replace the original 'fc' layer with a new Sequential block.
num_features = ResNet50_model_244.fc.in_features # Get the input dimension (2048)

ResNet50_model_244.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 5) # Output 5 classes for retinopathy levels
)

# 4️⃣ Move model to GPU if available
# We use .is_available() to check if a NVIDIA GPU is present
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ResNet50_model_244 = ResNet50_model_244.to(device)

# 5️⃣ Define Optimizer and Loss Function
# Only optimize the parameters of the new 'fc' layer
optimizer = optim.Adam(ResNet50_model_244.fc.parameters(), lr=1e-4)

# CrossEntropyLoss is the PyTorch equivalent of categorical_crossentropy
criterion = nn.CrossEntropyLoss()

print(f"Model successfully moved to: {device}")

# 5️⃣ Define Optimizer and Loss Function
# Only optimize the parameters of the new 'fc' layer (where requires_grad=True)
optimizer = optim.Adam(ResNet50_model_244.fc.parameters(), lr=1e-4)

# nn.CrossEntropyLoss in PyTorch expects raw scores (logits),
# so we don't need a Softmax layer at the end of our model.
criterion = nn.CrossEntropyLoss()

print(f"Model moved to: {device}")

If the code is correct you should see something _similar_ to the following output
```text
Model successfully moved to: cuda
Model moved to: cuda
```

# Example 3 - Step 10: Train Neural Network

The code in the cell below trains the neural network `ResNet50_model_244` for the number of epochs specified by the variable `EPOCHS`. A second variable, `PATIENCE`, controls the Early Stopping monitor. To keep training time reasonable the variable EPOCH has been set to 10 and the variable PATIENCE has been set to 3.

**TIME ALERT!**

We are analyzing a **LARGE** image dataset so training will take time. Even using Google fastest GPU hardware acceleration, it will probably take more than **1 hour to complete**. So don't start training if you can't afford to wait.

**DON'T WASTE YOUR MONEY!**

To get feedback on how long the training takes, the code in the cell below contains a "timer function". Specifically, the last line of code reads:

```text
print(f"Elapsed time: {hms_string(elapsed_time)}")
```
This code snippet prints out how long the training required.

If you weren't careful and skipped over running the cell that defined the custom function `hms_string()` at the start of this lesson, your model will train for an hour but **FAIL** at the end with an error message.

Since every time you use a Colab GPU **costs you money**, don't waste your time and _money_ by running this code cell if you didn't run _all_ of the code cells above before you run the next one.

**You have been warned.**

In [ ]:
# @title Example 3 - Step 10: Train Neural Network

import time
import torch
from tqdm.auto import tqdm

# 1️⃣ Initialize History Dictionary
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

EPOCHS = 10
PATIENCE = 3
best_val_loss = float('inf')
epochs_no_improve = 0

# Scheduler for learning rate
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

print(f"-- Training started for {EPOCHS} epochs ----------------------------")
start_time = time.time()

for epoch in range(EPOCHS):
    # --- TRAINING PHASE ---
    ResNet50_model_244.train()
    train_loss, train_correct = 0.0, 0
    total_train = 0

    for inputs, labels in tqdm(eg_train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = ResNet50_model_244(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_correct += torch.sum(preds == labels.data).item()
        total_train += labels.size(0)

    # --- VALIDATION PHASE ---
    ResNet50_model_244.eval()
    val_loss, val_correct = 0.0, 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in eg_val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = ResNet50_model_244(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += torch.sum(preds == labels.data).item()
            total_val += labels.size(0)

    # 2️⃣ Calculate and Append Metrics to History
    epoch_train_loss = train_loss / total_train
    epoch_train_acc = train_correct / total_train
    epoch_val_loss = val_loss / total_val
    epoch_val_acc = val_correct / total_val

    history['train_loss'].append(epoch_train_loss)
    history['train_acc'].append(epoch_train_acc)
    history['val_loss'].append(epoch_val_loss)
    history['val_acc'].append(epoch_val_acc)

    print(f"Epoch {epoch+1}: Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.4f} | "
          f"Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}")

    # 3️⃣ Early Stopping & Best Model Saving
    scheduler.step(epoch_val_loss)

    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        # Save the best weights locally
        torch.save(ResNet50_model_244.state_dict(), 'best_model_local.pth')
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

# ---------------------------------------------------------------------------
# 9️⃣ Inspect training
# ------------------------------------------------------------------------
print(f"\nTraining finished.")

# Compute elapsed time
elapsed_time = time.time() - start_time

# Print elapsed time
print(f"Elapsed time: {hms_string(elapsed_time)}")

# Print final training statistics
print(f"Final training completed in {epoch+1} epochs")

If the code is correct you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image03A.png)

Using an older, less powerful GPU accelerator (`T4 High-RAM`) training for 10 epochs required about 20 min to complete.


# Example 3 - Step 11: Plot Training/Validation Accuracy

A **Training-versus-Validation Accuracy Plot** provides a quick visual gauge of how well a model is learning and generalizing. As training progresses, a rising training accuracy shows that the model is fitting the data, while the validation accuracy tracks performance on unseen samples.

When both curves rise together and plateau, the model is likely well-balanced. If training accuracy climbs while validation accuracy lags or dips, the model is over-fitting; conversely, if both stay low, the model is under-fitting or too simple.

The shape of the curves also reveals training issues—sharp jumps or oscillations can signal an inappropriate learning rate, and noisy validation performance may indicate label noise or class imbalance. Thus, the plot informs decisions about early stopping, regularisation, architecture changes, or hyper-parameter tuning.


#### **Interpreting Common Patterns**

| Curve Shape | Interpretation | Suggested Action |
|-------------|----------------|------------------|
| **Both curves rise together and plateau at high accuracy** | Good fit & generalisation. | Continue training if you want to squeeze a bit more. |
| **Training rises, validation rises then drops** | Over‑fitting. | Add regularisation, dropout, data augmentation, or stop early. |
| **Both curves rise slowly and stay low** | Under‑fitting. | Increase model capacity, train longer, or reduce regularisation. |
| **Validation lags behind training by a fixed margin, but both improve** | Model learns but generalises less well. | Consider a larger training set, better augmentation, or a different architecture. |
| **Validation fluctuates wildly** | High variance due to small batch or high learning rate. | Reduce learning rate, increase batch size, or use a learning‑rate scheduler. |



In [ ]:
# @title Example 3 - Step 11: Plot Training/Validation Accuracy

import numpy as np
import matplotlib.pyplot as plt

# Pull metrics from history dictionary
eg_val_acc = [acc.cpu().item() if torch.is_tensor(acc) else acc for acc in history['val_acc']]
eg_train_acc = [acc.cpu().item() if torch.is_tensor(acc) else acc for acc in history['train_acc']]

# --- Find the epoch with the highest validation accuracy -------------
best_epoch_idx = np.argmax(eg_val_acc)
best_epoch_num = best_epoch_idx + 1

# -----------------------------------------------------------------------

plt.figure(figsize=(8, 4))
plt.plot(eg_val_acc, label='Validation Accuracy', marker='o')
plt.plot(eg_train_acc, label='Training Accuracy', marker='x')

# Vertical line at the best epoch
plt.axvline(best_epoch_idx, color='r', linestyle='--',
            label=f'Best epoch (epoch {best_epoch_num})')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training / Validation Accuracy (ResNet50)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

# Annotate the exact accuracy value at the best epoch
best_eg_val_acc = eg_val_acc[best_epoch_idx]
plt.text(best_epoch_idx, best_eg_val_acc,
         f' {best_eg_val_acc:.4f}',
         va='bottom', ha='left', color='r', fontweight='bold')

plt.tight_layout()
plt.show()

If the code is correct you should see something _similar_ to the following graph.

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image04A.png)


# Example 3 - Step 12: Save Model to GDrive

Run the next cell to save your retrained `ResNet50_model_244` model to your GDrive.

**IMPORTANT NOTE** You will be using this saved PyTorch model in a later class lesson so make sure **not** to delete it from your GDrive!

In [ ]:
# @title Example 3 - Step 10: Save Model Weights to Google Drive

import os
import torch

# --------------------------------------------------------------
# 1️⃣ Mount Google Drive (if not already done)
# --------------------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')

# --------------------------------------------------------------
# 2️⃣ Define the names / paths
# --------------------------------------------------------------
model_name  = "ResNet50_model_244"
gdrive_dir  = f"/content/drive/My Drive/{model_name}"  # Folder on Drive
# PyTorch convention uses .pth or .pt
gdrive_file = os.path.join(gdrive_dir, f"{model_name}.pth")

# --------------------------------------------------------------
# 3️⃣ Make sure the Drive folder exists
# --------------------------------------------------------------
os.makedirs(gdrive_dir, exist_ok=True)

# --------------------------------------------------------------
# 4️⃣ Save the weights *on* Drive
# --------------------------------------------------------------
# We save the state_dict, which is the dictionary of weights
torch.save(ResNet50_model_244.state_dict(), gdrive_file)

# --------------------------------------------------------------
# 5️⃣ Verify and List
# --------------------------------------------------------------
if os.path.exists(gdrive_file):
    print(f"Success! Model weights saved to Drive: {gdrive_file}")
else:
    print("Error: File not found. Check Drive connection.")

# List the directory contents
!ls -lh "/content/drive/My Drive/ResNet50_model_244"

If the code is correct you should see something similar to the following output
```text
Success! Model weights saved to Drive: /content/drive/My Drive/ResNet50_model_244/ResNet50_model_244.pth
total 92M
-rw------- 1 root root 92M Mar  1 02:40 ResNet50_model_244.pth
```

------------------------------------------


# **EXERCISE 3**

In **Exercise 3** you are going to re-analyze the Diabetic Retinopathy data set using `ResNet101` in conjunction with larger retinal images that are 512 X 512 pixels.

**`ResNet-101`** was built to give a **deeper**, **larger-receptive-field** backbone that can be trained stably thanks to residual connections. That extra depth—and the corresponding increase in receptive field and representational capacity—makes the network particularly suited to the high-resolution, high-complexity images that dominate modern clinical imaging datasets (e.g., whole-slide pathology, high‑res CT/MRI, retinal photographs).  

#### **1. Depth matters for large, complex images**

| Aspect                       | Why it matters in clinical imaging                                                                                     |
|------------------------------|------------------------------------------------------------------------------------------------------------------------|
| **Receptive field**         | • Deeper network ⇒ larger receptive field.<br>• Captures micro‑details (nuclei, capillaries) *and* macro‑context (lobule shape). |
| **Hierarchical feature extraction** | • Early layers learn edges & textures.<br>• Deeper layers combine them into complex motifs.<br>• Needed for multi‑level abstraction (malignant vs. normal). |
| **Parameter efficiency**    | • Residual blocks add depth without exploding the number of weights.<br>• 101‑layer stack packs more power while staying manageable. |


#### **2. Residual connections: the training engine that makes depth feasible**

* **Vanishing / exploding gradients** are especially problematic when you try to back‑propagate through dozens of nonlinear layers. Residual connections add a *shortcut* that lets the gradient flow almost unchanged from the output back to the input, so deeper models can be trained from scratch (or fine‑tuned on a new domain) without catastrophic loss of signal.
* In medical-image research, practitioners often *fine-tune* ImageNet‑pretrained models on limited clinical data. Because residual connections preserve gradient flow, a 101‑layer network can be adapted with fewer epochs and fewer data points than a comparable plain CNN.

#### **3. ResNet‑101 and typical clinical datasets**

| Dataset type              | Typical image size                                    | Typical complexity                                   | ResNet‑101 benefits                                                                                                                                                 |
|---------------------------|-------------------------------------------------------|-------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Whole‑slide pathology     | 10k‑40k px × 10k‑40k px (usually tiled to 512×512)   | Ultra‑fine texture + large‑scale architecture        | • Deep residual layers capture both micro‑ and macro‑features.<br>• Receptive field ≈ 400 px after 5 stages                                                      |
| Chest CT                  | 512×512 or larger                                    | Subtle texture changes, large organs                 | • Deeper encoder gives more context for slice‑to‑slice variation                                                                                                      |
| Retinal fundus            | 5000×5000 px (often cropped to 1024×1024)            | Fine vascular patterns + large vessels               | • Extra depth boosts sensitivity to minute capillary changes                                                                         
> **Bottom line**: ResNet‑101 was *not* invented *solely* for medical imaging, but its architectural traits (depth, residual links, large receptive field) make it a natural fit for the high‑resolution, highly variable images found in clinical datasets.

#### **4. Practical use-cases that illustrate the point**

1. **Whole‑slide analysis**  
   *U‑Net + ResNet‑101 encoder* is the standard backbone for segmentation of tumor vs. normal tissue. The encoder’s depth allows the network to produce rich multi‑scale feature maps that are essential for accurate boundary delineation.

2. **Histopathology classification**  
   A 101‑layer residual network fine‑tuned on 2048×2048 patches yields a 2‑fold improvement in AUC over a 50‑layer variant, especially on datasets where subtle nuclear pleomorphism is the key discriminant.

3. **Radiology reporting**  
   ResNet‑101 pre‑trained on ImageNet is used as the feature extractor for a downstream transformer that produces radiology reports. The larger depth improves the semantic quality of the extracted features, translating into more coherent, clinically relevant reports.

#### **5. What the “larger image size” means**

When people say ResNet‑101 was created “in part to analyze larger image sizes,” they're really referring to two intertwined design goals:

1. **Scale-aware receptive fields** - As the input resolution grows, the network’s receptive field must grow proportionally so that each unit still sees a meaningful portion of the image. Deeper networks naturally achieve this.

2. **Robust training** - High-resolution images contain many more pixels, which can amplify the effects of noise, small mis-alignments, or class imbalance. Residual learning keeps gradients stable, allowing the model to learn meaningful patterns from a massive pixel set without **over-fitting** or getting stuck.

---

#### **TL;DR**

ResNet‑101’s 101 layers give it a large receptive field and the capacity to learn complex, hierarchical features, which is vital for interpreting the fine‑grained details and global context in large medical images. Residual connections make training such a deep network feasible, allowing it to be fine‑tuned on the often limited clinical datasets that are typical in healthcare. That combination is why the architecture was chosen, and later adopted, for many high‑resolution clinical image‑analysis tasks.

#### **Image Size Comparison**

You should realize that `512×512×3` image is more than **4X larger** than a `244×244×3` image. Here's the math:

\begin{aligned}
512 \times 512 \times 3 &= 786{,}432 \\
244 \times 244 \times 3 &= 178{,}608 \\
\frac{786{,}432}{178{,}608} &\approx 4.4
\end{aligned}


So, the **512x512x3** image is approximately **4.4 times larger** than the **244×244×3** image in terms of pixel data. That additional detail might be critical for "seeing" small differences between the images in different diabetic retinal classes.

### **Exercise 3 - Step 1: Set ENVIRONMENTAL VARIABLES**

In the cell below write the code to define a your `ENVIRONMENTAL VARIABLES`. It is important to make the all of the following changes exactly as suggested.

**Code Hints:**

1. Change this line of code
```text
DOWNLOAD_SOURCE = URL+"diabetic_retinopathy_train_244.zip"
```
to read
```text
DOWNLOAD_SOURCE = URL+"diabetic_retinopathy_train_512.zip"
```
2. Change this line of code
```text
EXTRACT_TARGET = os.path.join(PATH,"retinopathy_244")
```
to read
```text
EXTRACT_TARGET = os.path.join(PATH,"retinopathy_512")
```
3. Change this line of code
```text
SOURCE = os.path.join(EXTRACT_TARGET, "train_244")
```
to read
```text
SOURCE = os.path.join(EXTRACT_TARGET, "train_512")
```



In [ ]:
# @title Exercise 3 - Step 1: Set ENVIRONMENTAL VARIABLES




If the code is correct you should see the following output

```text
DOWNLOAD_SOURCE= https://biologicslab.co/BIO1173/data/diabetic_retinopathy_train_512.zip
DOWNLOAD_NAME= diabetic_retinopathy_train_512.zip
EXTRACT_TARGET= ./retinopathy_512
SOURCE= ./retinopathy_512/train_512
ENVIRONMENTAL VARIABLES were successfully created.
```

# **Exercise 3 - Step 2: Download and Extract Image Data**

In the cell below write the code to (1) create the necessary directories, (2) download the Zip file containing the image data, (3) extract this data into the appropiate folder and (4) verify the sucess

**Code Hints:**

You can re-use the code in Example 3 - Step 2 _without_ making any code changes.

**TIME WARNING:** This Zip file is _substantially_ larger than the one you downloaded in Example 3. You can expect it to take approximately **7 minutes** to download and unzip the image files.

In [ ]:
# @title Exercise 3 - Step 2: Download and Extract Image Data



If the code is correct, you should see the following output:

```text
Creating necessary directories...done.
Downloading diabetic_retinopathy_train_512.zip...done.
Extracting diabetic_retinopathy_train_512.zip to ./retinopathy_512...done.
Verifying Extraction...
Successfully extracted 2 items:
  - trainLabels.csv (465317 bytes)
  - train_512 (directory)
Directory contents:
total 1040
drwxr-xr-x 3 root root   4096 Mar  1 02:49 .
drwxr-xr-x 1 root root   4096 Mar  1 02:42 ..
drwxr-xr-x 2 root root 581632 Mar  1 02:49 train_512
-rw-rw---- 1 root root 465317 Sep 12 15:05 trainLabels.csv

Extraction completed successfully!
```

# **Exercise 3 - Step 3: Load Labels for the Training Set**

In the cell below write the code to extract the label information from the file `trainLabel.csv` located in your `./retinopathy_512` folder and create a DataFrame called `ex_raw_df`.

**Code Hints:**

Copy the code from `Example 3 - Step 3` and then change the prefix `eg_` to `ex_` **everywhere** in the code.

In [ ]:
# @title Exercise 3 - Step 3: Load Labels for the Training Set


If the code is correct, you should see the following output:

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image05A.png)

# **Exercise 3 - Step 4: Validate Images**

In the cell below write the code to check whether an actual retinal image file exists for each image label in the DataFrame `ex_raw`.  

**Code Hints:**

Make sure to change the prefix `eg_` to `ex_` in the print statements are the bottom of the code copied from Example 3 - Step 4.

In [ ]:
# @title Exercise 3 - Step 4: Validate Images



If the code is correct you should see the following output

```text
Total images in DataFrame: 35126
Images that exist: 17448
Missing images: 17678
Valid DataFrame size: 17448
```

# **Exercise 3 - Step 5: Split Images into Training and Validation Sets**

In the cell below write the code to split your retinal images into a training set and a validation set with 80% of the images going into the training set. After splitting, print out the number images in both sets and a short sample of both the training set and the validation set.

**Code Hints:**

Copy the code from `Example 3 - Step 5` and change the prefix `eg_` to `ex_`.

In [ ]:
# @title Exercise 3 - Step 5: Split Images into Training and Validation Sets




If the code is correct you should see the following output

```text
Training set size   : 13958
Validation set size : 3490
Calculated split fraction = 0.7999770747363595

Sample training rows:
                 image level
16508   20710_left.png     0
8057   10109_right.png     2
23120   29217_left.png     0
24         31_left.png     0
12902   16217_left.png     0

Sample validation rows:
           image level
1   10_right.png     0
7   16_right.png     4
10   19_left.png     0
13  20_right.png     0
15  21_right.png     0
```

# **Exercise 3 - Step 6: Final Check for Valid Images**

In the cell below write the to make a final check that all the image files are valid.

**Code Hints:**

Make sure to change the prefix `eg_` to `ex_` every where in the code that you copied from Example 3 - Step 6.

In [ ]:
# @title Exercise 3 - Step 6: Final Check for Valid Images




If the code is correct you should see the following output

```text
Total files: 13958
Valid files: 13958
Invalid files: 0
After cleaning: 13958 samples
Total files: 3490
Valid files: 3490
Invalid files: 0
After cleaning: 3490 samples
```

# **Exercise 3: Step 7: Create Image Transforms and DataLoaders**

In the cell below write the code to create the image transforms and the dataloaders.

**Code Notes:**

1. Is is _very_ important that you change your image size.

Specifically you must change this code snippet:
```python
    #------------------------------------------------------------------
    # 0️⃣ IMPORTANT: Specify image size
    #-----------------------------------------------------------------
    EG_IMG_W = 244
    EG_IMG_H = 244
```
to read as
```python
    #------------------------------------------------------------------
    # 0️⃣ IMPORTANT: Specify image size
    #-----------------------------------------------------------------
    EX_IMG_W = 512
    EX_IMG_H = 512
```
The main objective of **Exercise 3** is to analyze larger retinal imagee (512 X 512 pixels) to see if that will improve classification accuracy.

2. Change batch name

Specifically you must change this code snippet:
```python
    # Specify batch size
    EG_BATCH_TRAIN  = 64
    EG_BATCH_VAL    = 64
```
to read as
```python
    # Specify batch size
    EX_BATCH_TRAIN  = 64
    EX_BATCH_VAL    = 64
```
You will also need to change these values in the code block.


3. Make sure to change the prefix `eg_` to `ex_` as well as `EG_` to `EX_` everywhere they appear in the code block.

In [ ]:
# @title Exercise 3: Step 7: Create Image Transforms and DataLoaders




If the code is correct you should see the following output

```text
TRAIN batch images: torch.Size([64, 3, 512, 512])
TRAIN batch labels: torch.Size([64])
VAL batch images:   torch.Size([64, 3, 512, 512])
VAL batch labels:   torch.Size([64])
```

# **Exercise 3 - Step 8: Check Class Distribution**

In the cell below write the code to generate a bar graph showing the number of images in each of the 5 output classes.

**Code Hints:**

Copy the code from `Example 3 - Step 6` and change the prefix `eg_` to `ex_` everywhere that it occurs.



In [ ]:
# @title Exercise 3 - Step 8: Check Class Distribution



If the code is correct you should see the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image07A.png)

```text
Class 0 (0): 1602 samples
Class 1 (1): 516 samples
Class 2 (2): 1055 samples
Class 3 (3): 175 samples
Class 4 (4): 142 samples
```
As before, the classes are not balanced.

# **Exercise 3 - Step 9: Setup `ResNet101` Base Network**

In the cell below write the code to setup your base network using `ResNet101` instead of `ResNet50`. Called your new model `ResNet101_model_512`.

**Code Hints:**

1. Change this line of code
```python
    # 1️⃣ Load the base ResNet101 model with pre-trained weights
    weights = models.ResNet50_Weights.DEFAULT
```
to read
```python
    # 1️⃣ Load the base ResNet101 model with pre-trained weights
    weights = models.ResNet101_Weights.DEFAULT
```

2. Change this line of code
```python
    # 2️⃣ Freeze all layers in the base model
    for param in ResNet50_model_244.parameters():
```
to read
```python
    # 2️⃣ Freeze all layers in the base model
    for param in ResNet101_model_512.parameters():
```

Change `ResNet50_model_244` to `ResNet101_model_512` everywhere it occurs in the code block.


In [ ]:
# @title Exercise 3 - Step 9: Setup `ResNet101` Base Network



If the code is correct you should see the following output

```text
Model successfully moved to: cuda
Model moved to: cuda
```

# **Exercise 3 - Step 10: Train Neural Network**

In the cell below write the code to train your neural network `ResNet101_model_512`.

#### **Please Read this Carefully**

The `ResNet101_model_512` is perhaps the largest neural network that you will be asked to train in this course. Due to the large size of the images (512x512 pixels), training this model for even `10` epochs requires close to **2 hours!** So don't start training your model unless you have enough time.

**Code Hints:**

1. Change `ResNet50_model_244` to read `ResNet101_model_512` everywhere it occurs in the code block.

2. Change the prefix `eg_` to `ex_` everywhere it occurs in the code block.

In [ ]:
# @title Exercise 3 - Step 10: Train Neural Network




If the code is correct, you should see something _similar_ to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image08A.png)


# **Exercise 3 - Step 11: Plot Training History**

In the cell below write the code to plot the training history of your `ResNet101_model_512`.

Copy the code in Example 3 - Step 9 into the cell below.

**Code Hints:**

1. Change the prefix `eg_` to `ex_` everywhere in the copied code.
2. Change the plot title from `ResNet50` to `ResNet101`.


In [ ]:
# @title Exercise 3 - Step 11: Plot Training History



If the code is correct you should see something similar to the following output

![__](https://biologicslab.co/BIO1173/images/class_03/class_03_2_image09A.png)



# **Exercise 3 - Step 12: Save Model to GDrive**

In the next cell write the code to save your `ResNet101_model_512` to your GDrive.

**Code Hints:**

Change this line of code
```python
    model_name = "ResNet50_model_244" # model object name
```
to read
```python
    model_name = "ResNet101_model_512"  # model object name
```

In [ ]:
# @title Exercise 3 - Step 12: Save Model to GDrive

import os
import torch

# --------------------------------------------------------------
# 1️⃣ Mount Google Drive (if not already done)
# --------------------------------------------------------------
# from google.colab import drive
# drive.mount('/content/drive')

# --------------------------------------------------------------
# 2️⃣ Define the names / paths
# --------------------------------------------------------------
model_name  = "ResNet101_model_512"
gdrive_dir  = f"/content/drive/My Drive/{model_name}"  # Folder on Drive
# PyTorch convention uses .pth or .pt
gdrive_file = os.path.join(gdrive_dir, f"{model_name}.pth")

# --------------------------------------------------------------
# 3️⃣ Make sure the Drive folder exists
# --------------------------------------------------------------
os.makedirs(gdrive_dir, exist_ok=True)

# --------------------------------------------------------------
# 4️⃣ Save the weights *on* Drive
# --------------------------------------------------------------
# We save the state_dict, which is the dictionary of weights
torch.save(ResNet101_model_512.state_dict(), gdrive_file)

# --------------------------------------------------------------
# 5️⃣ Verify and List
# --------------------------------------------------------------
if os.path.exists(gdrive_file):
    print(f"Success! Model weights saved to Drive: {gdrive_file}")
else:
    print("Error: File not found. Check Drive connection.")

# List the directory contents
!ls -lh "/content/drive/My Drive/ResNet101_model_512"

If the code is correct you should see something similar to the following output
```text
Success! Model weights saved to Drive: /content/drive/My Drive/ResNet101_model_512/ResNet101_model_512.pth
total 165M
-rw------- 1 root root 165M Dec 29 08:31 ResNet101_model_512.pth
-rw------- 1 root root  14K Sep 11 00:27 test_label.csv
```

Your `ResNet101_model_512` is now safely stored on your GDrive.

If you wanted to, you could easily copy your model back from your GDrive into you current Google Colab directory and use it analyze new retinal fundus images, looking for signs of diabetic retinopathy. Since your model is already trained, you could analyze the retinal images from your new patient as he/she sits in your office.

Always keep in mind a common saying among members of the ML/AI community:

> **_“Train-once, deploy-many”_**

**IMPORTANT NOTE:** Make sure **not** to erase your saved file from your GDrive. You will need to use it later in a different class lesson.

# **Electronic Submission**

When you run the code in the cell below, it will grade your Colab notebook and tell you your pending grade as it currently stands. You will be given the choice to either submit your notebook for final grading or the option to continue your work on one (or more) Exercises. You no longer have the option to upload a PDF of your Colab notebook to Canvas for grading. Grant Access to your Colab Secrets if you are asked to do so.

**NOTE:** You grade on this Colab notebook will be based solely on the code in your **Exercises**. Failure to run one (or more) Examples will not affect your grade.

In [ ]:
# @title  Electronic Submission

import urllib.request
import ssl
import time

url = "https://biologicslab.co/BIO1173/backend_code/validate.py?v=" + str(time.time())

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

req = urllib.request.Request(
    url,
    headers={
        "Cache-Control": "no-cache, no-store, must-revalidate",
        "Pragma": "no-cache",
        "Expires": "0"
    }
)

with urllib.request.urlopen(req, context=ctx) as r:
    exec(r.read().decode("utf-8"))

main()

# **Additional Information**

### **The Diabetic Retinopathy Dataset**

The **Diabetic Retinopathy Dataset** used in this lesson was part of a 2015 Kaggle Competition.

Here is a summary of the winners and what they did to win this competition.

### Winner of the 2015 Kaggle Diabetic Retinopathy Detection competition
**Team**: *o_O* (Mathis Antony & Stephan Brüggemann)  
**Score**: 0.845 weighted quadratic‑weighted Kappa (private leaderboard)  
**Public leaderboard**: 7th place (5.8 k K‑appa)

| Item | Details | Source |
|------|---------|--------|
| **Winning team** | “o_O” (Mathis Antony & Stephan Brüggemann) | 5.8 k K‑appa on the private leaderboard, 7 th place on the public leaderboard【9†L18-L23】 |
| **Overall performance** | 0.845 weighted quadratic weighted Kappa (private leaderboard) | 5.8 k K‑appa, 0.845 score【9†L19-L22】 |
| **Core architecture** | Two custom 2‑D convolutional nets (Net A & Net B) with a **per‑patient blending network** (Table 2) | 13‑25 | 12‑15 |
| **Training framework** | Lasagne + nolearn (Theano) | 10‑11 |
| **Image size** | 128 × 128, 256 × 256 and 512 × 512 (large color images, cropped to remove background) | 21‑24 |
| **Pre‑training strategy** | *First* train a small network on 128‑pixel images.  Weights are then used to initialise an intermediate‑size network (trained on 256 px) and finally a 512‑pixel network.  Orthogonal initialization for all weights. | 105‑108 |
| **Data augmentation** | Translation, stretching, rotation, flipping, colour jitter; per‑channel zero‑mean/unit‑variance scaling; 112/224/448 output sizes for 128/256/512 input images. | 115‑121 |
| **Class imbalance handling** | Dynamic resampling: oversample rare classes initially, then gradually reduce; resampling weights \((1.36, 14.4, 6.64, 40.2, 49.6)\) → \((1,2,2,2,2)\). | 82‑96 |
| **Training schedule** | Nesterov momentum with a fixed learning‑rate schedule over 250 epochs; learning rates 0.003 (epoch 0) → 0.00003 (epoch 150); L2 weight decay 0.0005; dropout after convolution and dense layers. | 31‑36, 70‑73 |
| **Loss & objective** | Mean‑squared‑error regression (output thresholded at (0.5,1.5,2.5,3.5) to obtain integer grades). | 75‑77 |
| **Blending network** | Input: mean & std of the RMSPool layer over 50 augmentations for each eye (µ,σ) + eye‑side indicator; 8193‑input → Dense 32 → Maxout 16 → Dense 32 → Maxout 16; Adam optimiser with a schedule (5 e‑4 → 5 e‑7). | 148‑156, 158‑166 |
| **Final ensemble** | Average of the two conv‑net predictions, blended with the patient‑level network; score 0.845 (private) vs 0.824 (no per‑patient blend). | 167‑169 |
| **Key design choices that yielded the win** | 1. **Large input resolution** – 512 × 512 (and even 768 × 768 for 0.81 Kappa) to capture micro‑aneurysms. 2. **Stage‑wise pre‑training** – starting from 128 px to 512 px to stabilise training. 3. **Extensive data augmentation & per‑channel normalisation**. 4. **Dynamic resampling** to address class imbalance without a weighted loss. 5. **Per‑patient blending** that aggregates information from both eyes and multiple augmentations. 6. **Ensembling of two independently trained nets**. | 45‑69, 82‑96, 100‑108, 115‑121, 131‑139, 167‑169 |

### **Why their approach won**

The combination of **large‑resolution images**, staged pre‑training, aggressive augmentation, careful imbalance handling, and per‑patient feature blending allowed the o_O model to achieve the highest weighted quadratic‑weighted κ score in the 2015 competition.

